# Data Preparation

database used: https://mcauleylab.ucsd.edu/public_datasets/gdrive/googlelocal/

In [ ]:
import os, json, csv
os.sys.path.append("../src")

from objects import InputData
from data_processsing.data_loader import parse_csv, parse_json, save_to_json

DATA_DIR= "../data"
PROCESSED_DIR = f"{DATA_DIR}/processed"
RAW_DIR = f"{DATA_DIR}/raw"


In [ ]:
review_json_path=f"{RAW_DIR}/reviews-vermont.json"
business_info_path=f"{RAW_DIR}/meta-vermont.json"

output_path = f"{PROCESSED_DIR}/vermont_test_set.json"

selected_data = parse_json(review_json_path, 200, business_info_path)
save_to_json(selected_data, output_path)


print(selected_data[0])

## Data Labelling 
- manaul annotation:
- AI annotation

In [ ]:
csv_name = output_path.replace('.json', '_annotated.csv')

with open(output_path, 'r') as f:
    data = json.load(f)

annotation_fields = ['spam', 'relevance', 'credible', '_sentiment', '_informative']
with open(csv_name, 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=list(data[0].keys())+ annotation_fields)
   
    for item in data:
        for field in annotation_fields:
            if field not in item:
                item[field] = ''
        writer.writerow(item)
